In [1]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image as keras_image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import os

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [3]:
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  # Invalid device or cannot modify virtual devices once initialized.
  print("Failed!")


In [4]:
model = ResNet50(weights='imagenet')

In [5]:
import h5py

In [6]:
import urllib.request
if not os.path.exists('val_preds.h5'):
    print("Downloading MICP calibration data - be patient!")
    urllib.request.urlretrieve("https://cml.rhul.ac.uk/people/ptocca/ILSVRC2012-CP/val_preds.h5",
                               'val_preds.h5')

In [7]:
with h5py.File('val_preds.h5','r') as f:
    preds = f['preds'][:]

In [8]:
def pValues(calibrationAlphas,testAlphas,randomized=False):
    testAlphas = np.array(testAlphas)
    sortedCalAlphas = np.sort(calibrationAlphas)
    
    leftPositions = np.searchsorted(sortedCalAlphas,testAlphas)
    
    if randomized:
        rightPositions = np.searchsorted(sortedCalAlphas,testAlphas,side='right')
        ties  = rightPositions-leftPositions+1   # ties in cal set plus the test alpha itself
        randomizedTies = ties * np.random.uniform(size=len(ties))
        return  (len(calibrationAlphas) - rightPositions + randomizedTies)/(len(calibrationAlphas)+1)
    else:
        return  (len(calibrationAlphas) - leftPositions + 1)/(len(calibrationAlphas)+1)


In [9]:
 # shape: (num_labels,num_test_objects)

In [10]:
def micp_pValues(preds,test_preds,y_cal):
    ncm_cal = -preds
    ncm_test = -test_preds
    
    micp_pValues = []

    for i in range(test_preds.shape[1]):
        p_i = pValues(ncm_cal[:,i][y_cal==i],ncm_test[:,i])
        micp_pValues.append(p_i)

    micp_pValues = np.array(micp_pValues)
    
    return micp_pValues

In [11]:
# ilsrvc_dir = "/mnt/d/Research/ILSVRC2012/"
ilsrvc_dir = "."

In [12]:
gt_file = os.path.join(ilsrvc_dir,"ILSVRC2012_validation_ground_truth.txt")
lbls_file = os.path.join(ilsrvc_dir,"labels.txt")

In [13]:
mapping_file = os.path.join(ilsrvc_dir,"ILSVRC2012_mapping.txt")

In [14]:
n_to_ki = {}
ki_to_synset = {}
with open(os.path.join(ilsrvc_dir,'synset_words.txt')) as f:
    for i,l in enumerate(f):
        n_to_ki[l.split()[0].strip()]=i
        ki_to_synset[i]=l[10:].split(",")[0].strip()

In [15]:
ii_to_n = ["Error"]
with open(mapping_file) as f:
    for l in f:
        lf = l.split()
        ii_to_n.append(lf[1].strip())

In [16]:
ii_to_ki = [0]+[n_to_ki[ii_to_n[i]] for i in range(1,1001)]

In [17]:
ground_truth_ki = np.zeros(50000,dtype=np.int16)

In [18]:
with open(gt_file) as f:
    for i,l in enumerate(f):
        ground_truth_ki[i] = ii_to_ki[int(l)]

In [20]:
from ipywidgets import IntSlider,Image,interactive,VBox,Textarea,Layout,FloatSlider,HBox,Label,Output
import io

In [27]:
import PIL.Image


ModuleNotFoundError: No module named 'urllib.requests'

In [28]:
def show_pic(i,eps):
    #img_file = os.path.join(ilsrvc_dir,"img","ILSVRC2012_val_%08d.JPEG"%i)
    # with open(img_file,"rb") as f:
    #    img.value = f.read()
    url="""https://cml.rhul.ac.uk/people/ptocca/ILSVRC2012-CP/img/ILSVRC2012_val_%08d.JPEG"""%i

    lbl = ki_to_synset[ground_truth_ki[i-1]]

    desc.children[1].value = lbl

#    img_data = keras_image.load_img(img_file, target_size=(224, 224))
    img_data = PIL.Image.open(urllib.request.urlopen(url)).resize((224,224))
    output = io.BytesIO()
    img_data.save(output,format="PNG")
    img.value = output.getvalue()
    x = keras_image.img_to_array(img_data)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    test_preds = model.predict(x)
    top = ["%s: %0.3f"%(l[1],l[2]) for l in decode_predictions(test_preds, top=5)[0]]
    resnet50.children[1].value = "\n".join(top)

    p_vals = micp_pValues(preds,test_preds,ground_truth_ki)

    ps = np.argwhere(p_vals>eps)[:,0].T
    ps_p_vals = p_vals[ps].flatten()
    sorting_by_p_val = np.argsort(ps_p_vals)[::-1] 
    ps_synset = [ki_to_synset[k]+":%0.3f"%p for k,p in zip(ps[sorting_by_p_val],ps_p_vals[sorting_by_p_val])]

    CP.children[0].value = "Conformal Predictor at significance level %0.2f"%eps
    CP.children[1].value = "\n".join(ps_synset)
    return

In [33]:
desc = VBox([Label("True label"),
             Textarea("N/A",layout=Layout(height="100%"))])

img = Image(layout=Layout(height="400px",width="400px"))

resnet50 = VBox([Label("ResNet50 Probability (top 5)"),
                 Textarea(layout=Layout(height="100%"))])
CP = VBox([Label("Conformal Predictor at significance level 'eps'"),
           Textarea(layout=Layout(height="100%"))])
labels = HBox([desc,resnet50,CP],layout=Layout(height="200px",align_content="stretch"))

pic_idx = IntSlider(value=500, min=1,max=1000,continuous_update=False,layout=Layout(width="90%",align_items='center'))
eps_slider = FloatSlider(value=0.1,min=0.01,max=0.50,step=0.01,layout=Layout(width="90%",align_items='center'))
                    

gui = VBox([img,pic_idx,eps_slider,labels],layout=Layout(align_items='center'))

show_pic(500,0.1);


In [34]:
interactive(show_pic,i = pic_idx, eps = eps_slider)

gui